In [1]:
pwd

'e:\\projects of camp\\Machine Failure\xa0Prediction'

In [2]:
import pandas as pd
import numpy as np

data=pd.read_csv("E:\projects of camp\Machine Failure Prediction\data\data.csv")

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\H\AppData\Local\Temp\ipykernel_9800\268490379.py:4: SyntaxWarning: invalid escape sequence '\p'
  data=pd.read_csv("E:\projects of camp\Machine Failure Prediction\data\data.csv")


In [3]:
data.head()

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
0,0,7,7,1,6,6,36,3,1,1
1,190,1,3,3,5,1,20,4,1,0
2,31,7,2,2,6,1,24,6,1,0
3,83,4,3,4,5,1,28,6,1,0
4,640,7,5,6,4,0,68,6,1,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   footfall     944 non-null    int64
 1   tempMode     944 non-null    int64
 2   AQ           944 non-null    int64
 3   USS          944 non-null    int64
 4   CS           944 non-null    int64
 5   VOC          944 non-null    int64
 6   RP           944 non-null    int64
 7   IP           944 non-null    int64
 8   Temperature  944 non-null    int64
 9   fail         944 non-null    int64
dtypes: int64(10)
memory usage: 73.9 KB


In [5]:
# يظهر من خلال المعلومات السابقة انه لا يوجد قيم مفقودة في البياناتلذلك لا نحتاج الى التعامل مع القيم المفقودة

In [6]:
num_cols = ["footfall","USS","CS","VOC","RP","IP","Temperature","AQ"]
cat_cols=["tempMode"]
target="fail"

In [41]:
x=data.drop(columns=[target])
y=data[target]

In [42]:
y

0      1
1      0
2      0
3      0
4      0
      ..
939    1
940    1
941    1
942    1
943    1
Name: fail, Length: 944, dtype: int64

In [9]:
num_data=x[num_cols]
cot_data=x[cat_cols]

In [44]:
cot_data

,tempMode
0,7
1,1
2,7
3,4
4,7
...,...
939,7
940,7
941,3
942,6


In [11]:
data.head()
# من خلال النظر الى  النتيجة قيم الاعمدة غير متناسقة لذلك من الافضل استخدام standardization

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
0,0,7,7,1,6,6,36,3,1,1
1,190,1,3,3,5,1,20,4,1,0
2,31,7,2,2,6,1,24,6,1,0
3,83,4,3,4,5,1,28,6,1,0
4,640,7,5,6,4,0,68,6,1,0


In [12]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_data =pd.DataFrame(scaler.fit_transform(num_data),columns=num_cols)

In [13]:
scaled_data

,footfall,USS,CS,VOC,RP,IP,Temperature,AQ
0,-0.283153,-1.402480,0.477610,1.389813,-0.672788,-0.979504,-2.567407,1.860497
1,-0.107558,0.043660,-0.310613,-0.810763,-1.647540,-0.353894,-2.567407,-0.921775
2,-0.254504,-0.679410,0.477610,-0.810763,-1.403852,0.897327,-2.567407,-1.617343
3,-0.206446,0.766730,-0.310613,-0.810763,-1.160164,0.897327,-2.567407,-0.921775
4,0.308326,2.212869,-1.098836,-1.250878,1.276716,0.897327,-2.567407,0.469361
...,...,...,...,...,...,...,...,...
939,-0.283153,-1.402480,0.477610,0.509582,1.581326,0.897327,1.284147,1.860497
940,-0.283153,-0.679410,0.477610,1.389813,0.180120,0.897327,1.284147,0.469361
941,-0.283153,-0.679410,1.265832,0.949698,-0.246334,0.897327,1.284147,1.164929
942,-0.283153,-0.679410,-0.310613,1.389813,-0.063568,1.522937,1.284147,1.164929


In [14]:
data.describe()
#  يظهر من خلال الوصف السابق ان البيانات متوازنة ولا يوجد بها قيم شاذة

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
count,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000
mean,306.381356,3.727754,4.325212,2.939619,5.394068,2.842161,47.043432,4.565678,16.331568,0.416314
std,1082.606745,2.677235,1.438436,1.383725,1.269349,2.273337,16.423130,1.599287,5.974781,0.493208
min,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,19.000000,1.000000,1.000000,0.000000
25%,1.000000,1.000000,3.000000,2.000000,5.000000,1.000000,34.000000,3.000000,14.000000,0.000000
50%,22.000000,3.000000,4.000000,3.000000,6.000000,2.000000,44.000000,4.000000,17.000000,0.000000
75%,110.000000,7.000000,6.000000,4.000000,6.000000,5.000000,58.000000,6.000000,21.000000,1.000000
max,7300.000000,7.000000,7.000000,7.000000,7.000000,6.000000,91.000000,7.000000,24.000000,1.000000


In [19]:
import pickle 
with open("E:\projects of camp\Machine Failure Prediction\data\scaler.pkl","wb") as f:
    pickle.dump(scaler,f)


<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\H\AppData\Local\Temp\ipykernel_9800\3826135728.py:2: SyntaxWarning: invalid escape sequence '\p'
  with open("E:\projects of camp\Machine Failure Prediction\data\scaler.pkl","wb") as f:


In [50]:
y = pd.DataFrame(y)

In [51]:
y

,fail
0,1
1,0
2,0
3,0
4,0
...,...
939,1
940,1
941,1
942,1


In [54]:
data_precossed = pd.concat([scaled_data ,cot_data, y], axis=1)


In [55]:
data_precossed.to_csv("E:\projects of camp\Machine Failure Prediction\data\prepro_data.csv",index=False)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\H\AppData\Local\Temp\ipykernel_9800\1022475774.py:1: SyntaxWarning: invalid escape sequence '\p'
  data_precossed.to_csv("E:\projects of camp\Machine Failure Prediction\data\prepro_data.csv",index=False)
